# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

C. Desgrange  ->  C. Desgrange  |  ['C. Desgrange']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 59 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2305.19355


extracting tarball to tmp_2305.19355...

 done.


C. Desgrange  ->  C. Desgrange  |  ['C. Desgrange']


Found 83 bibliographic references in tmp_2305.19355/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.19355-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.19355) | **Trade-offs in high-contrast integral field spectroscopy for exoplanet  detection and characterisation: Young gas giants in emission**  |
|| R. Landman, et al. -- incl., <mark>C. Desgrange</mark> |
|*Appeared on*| *2023-06-01*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Context: Combining high-contrast imaging with medium- or high-resolution integral field spectroscopy has the potential to boost the detection rate of exoplanets, especially at small angular separations. Furthermore, it immediately provides a spectrum of the planet that can be used to characterise its atmosphere. The achievable spectral resolution, wavelength coverage, and FOV of such an instrument are limited by the number of available detector pixels. Methods: The trade-offs are studied through end-to-end simulations of a typical high-contrast imaging instrument, analytical considerations, and atmospheric retrievals. The results are then validated with archival VLT/SINFONI data of the planet beta Pictoris b. Results: We show that molecular absorption spectra generally have decreasing power towards higher spectral resolution and that molecule mapping is already powerful for moderate resolutions (R>300). When choosing between wavelength coverage and spectral resolution for a given number of spectral bins, it is best to first increase the spectral resolution until R~2,000 and then maximise the bandwidth within an observing band. We find that T-type companions are most easily detected in the J/H band through methane and water features, while L-type companions are best observed in the H/K band through water and CO features. Such an instrument does not need to have a large FOV, as most of the gain in contrast is obtained in the speckle-limited regime close to the star. We show that the same conclusions are valid for the constraints on atmospheric parameters such as the C/O ratio, metallicity, surface gravity, and temperature, while higher spectral resolution (R~10,000) is required to constrain the radial velocity and spin of the planet. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2305.19355.md
    + _build/html/tmp_2305.19355/./PSD_hr8799e_new_appendix=False_both.png
    + _build/html/tmp_2305.19355/./fov_vs_R_tradeoff_51eri_J_2000pix.png
    + _build/html/tmp_2305.19355/./fov_vs_R_tradeoff_beta_pic_H_2000pix.png
    + _build/html/tmp_2305.19355/./fov_vs_B_tradeoff_51eri_J_2048pix_2.0.png
    + _build/html/tmp_2305.19355/./fov_vs_B_tradeoff_beta_pic_H_2048pix_2.0.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\appropto}{\mathrel{\vcenter{$
$  \offinterlineskip\halign{\hfil##\cr$
$    \propto\cr\noalign{\kern2pt}\sim\cr\noalign{\kern-2pt}}}}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.8}$</div>



<div id="title">

# Trade-offs in high-contrast integral field spectroscopy for exoplanet detection and characterisation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2305.19355-b31b1b.svg)](https://arxiv.org/abs/2305.19355)<mark>Appeared on: 2023-06-01</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

R. Landman, et al. -- incl., <mark>C. Desgrange</mark>

</div>
<div id="abstract">

**Abstract:** Combining high-contrast imaging with medium- or high-resolution integral field spectroscopy has the potential to boost the detection rate of exoplanets, especially at small angular separations. Furthermore, it immediately provides a spectrum of the planet that can be used to characterise its atmosphere. The achievable spectral resolution, wavelength coverage, and FOV of such an instrument are limited by the number of available detector pixels. We aim to study the effect of the spectral resolution, wavelength coverage, and FOV on the detection and characterisation potential of medium- to high-resolution integral field spectrographs with molecule mapping. The trade-offs are studied through end-to-end simulations of a typical high-contrast imaging instrument, analytical considerations, and atmospheric retrievals. The results are then validated with archival VLT/SINFONI data of the planet $\beta$ Pictoris b. We show that molecular absorption spectra generally have decreasing power towards higher spectral resolution and that molecule mapping is already powerful for moderate resolutions (R $\gtrsim$ 300). When choosing between wavelength coverage and spectral resolution for a given number of spectral bins, it is best to first increase the spectral resolution until R $\sim$ 2,000 and then maximise the bandwidth within an observing band. We find that T-type companions are most easily detected in the J/H band through methane and water features, while L-type companions are best observed in the H/K band through water and CO features. Such an instrument does not need to have a large FOV, as most of the gain in contrast is obtained in the speckle-limited regime close to the star. We show that the same conclusions are valid for the constraints on atmospheric parameters such as the C/O ratio, metallicity, surface gravity, and temperature, while higher spectral resolution (R $\gtrsim$ 10,000) is required to constrain the radial velocity and spin of the planet.

</div>

<div id="div_fig1">

<img src="tmp_2305.19355/./PSD_hr8799e_new_appendix=False_both.png" alt="Fig1" width="100%"/>

**Figure 1. -** Normalised PSD of different contributions to the observed spectrum between 1 and 2.5 $\mu$m. Here, we assumed a planet with an effective temperature of 1200 K and log(g) of 4.0. The panels show the results without rotational broadening (top) and with rotational broadening of $v_{\textrm{rot}}\sin i = 15$ km/s (bottom). (*fig:psd*)

</div>
<div id="div_fig2">

<img src="tmp_2305.19355/./fov_vs_R_tradeoff_51eri_J_2000pix.png" alt="Fig10.1" width="50%"/><img src="tmp_2305.19355/./fov_vs_R_tradeoff_beta_pic_H_2000pix.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Contrast curves with molecule mapping from the simulated observations. Different lines are for different combinations of spectral resolution and FOVs, and all use the same number of detector pixels. The inset shows the residuals after applying ADI on the mock observations. **Left:** 51 Eri b in the J-band with a 5\% spectral bandwidth. **Right:**$\beta$ Pic b in the H-band with a 10\% bandwidth.  (*fig:contrast_curves*)

</div>
<div id="div_fig3">

<img src="tmp_2305.19355/./fov_vs_B_tradeoff_51eri_J_2048pix_2.0.png" alt="Fig11.1" width="50%"/><img src="tmp_2305.19355/./fov_vs_B_tradeoff_beta_pic_H_2048pix_2.0.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Same as Fig. \ref{fig:contrast_curves} but the spectral resolution is fixed at R=8,000 while varying the spectral bandwidth. (*fig:contrast_curves*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2305.19355"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

212  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
